# Export raw tick data using dedicated service - C#

### Overview
This sample demonstrates how to request from a dedicated raw data service **on-demand** tick by tick data `for a given instrument or a given watchlist`. 

### Inputs/outputs
Tick data extraction sample requires instrument's identifier, date time intervals as per inputs and exports tick by tick data file. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick dataset from the hosted service. The queried endpoint in this script are:
* *TickRawService*: to directly retrieve raw dataset from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run tick by tick dataset extraction sample

### Step 1: Install packages

In [ ]:
#r "nuget: Systemathics.Apis"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using System.IO;
using System.Globalization;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request
To request *raw tick data* service, we need to specify instrument identifier and a period of time. The following code snippets ensure required objects are created and proceed to request instantiation.

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
var start = new DateTime(2021, 03, 05);
var end = new DateTime(2021, 03, 05);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
     StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
     EndTime = new TimeOfDay { Hours = 14, Minutes = 10, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous space selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create the market data request and instantiate the service:

In [ ]:
// Generate raw tick data request
var request = new TickRawRequest() 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints,
    Level = Level.TradesAndBook
};

// Instantiate the raw tick data service
var service = new TickRawService.TickRawServiceClient(channel);

In [ ]:
var reply = service.TickRaw(request, headers);

### Step 4: Retrieve tick by tick data

Tick by tick data is sent through a *stream*, the following code snippet exports tick by tick dataset in a file in the same folder as this sample:

In [ ]:
// Get the tick raw data
var call = service.TickRaw(request, headers);

// Export tick raw data to a txt file
var ticksCount =0;
using (var fs = new StreamWriter(File.OpenWrite($"{ticker}_raw_ticks.txt")))
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    var items = current.RawItems;
    
    if (items != null) 
    {
        foreach (var item in items)
        {
            ticksCount++; 
            var ss = Encoding.ASCII.GetString(item.Payload.ToByteArray()).Substring(2);
            fs.WriteLine($"{item.TimeStamp.ToDateTime():O} {ss}");
        }
    }
}

The following code snippet is a summary with some metrics about the dateset export: 

In [ ]:
Console.WriteLine($"Start date: {start:yyyy-MM-dd}");
Console.WriteLine($"Start date: {end:yyyy-MM-dd}");
Console.WriteLine($"Total ticks retrieved for {ticker}: {ticksCount:N0}");